In [1]:
from PIL import Image
import os.path
import numpy as np
import itertools
import keras.backend as K
import keras
from IPython.display import clear_output
import pandas
import time

/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
def resizeImg(action, x , y):
    avgX = x
    avgY = y
    counter = 0
    filelist= [file for file in os.listdir(action + '/') if file.endswith('.jpeg')]
    for filename in filelist:
    #for filename in itertools.islice(filelist, 0, 100):
        img = Image.open(action + '/' + filename)
        imgResize = img.resize((avgX, avgY), Image.LANCZOS)
        imgResize.save('resized1500x1000/' + action + '/' + filename)
        img.close()
        counter +=1
        if (counter % 100) == 0:
            clear_output(wait=True)
            print(counter, end =" ") 
        

In [3]:
resizeImg('train',1500,1000)

35100 

In [ ]:
def process_image(action):
    counter = 0
    labels = pandas.read_csv('trainLabels.csv')
    X = []
    y = []

    filelist= [file for file in os.listdir('resized375x250/' + action + '/') if file.endswith('.jpeg')]
    for filename in filelist:
    #for filename in itertools.islice(filelist, 0, 15):
        #print(filename)
        img = Image.open('resized375x250/' + action + '/' + filename)
        #imgData = list(img.getdata())
        #imgData = np.asarray(imgData)
        #imgData = img.getdata()
        #imgData = np.asarray(imgData)
        rgbimg = Image.new("RGB", img.size)
        rgbimg.paste(img)
        imgData = np.asarray(rgbimg)
        #imgData = np.asarray(img)
        #print(imgData.shape)
        #print(imgData.ndim)
        if imgData.ndim == 3:
            x = imgData.reshape(375,250,3)
            X.append(x)
            level = labels.loc[labels['image'] == filename[:-5]]
            y.append(level['level'].values)
        img.close()
        counter +=1
        if (counter % 100) == 0:
            clear_output(wait=True)
            print(counter, end =" ") 
        
    return X, y

In [ ]:
start_time = time.time()
X, y = process_image('train')
elapsed_time = time.time() - start_time
print(elapsed_time)

In [ ]:
X = np.asarray(X)
y = np.asarray(y)
print("shape of X: " + str(X.shape))
print("length of X: " + str(len(X)))
print("shape of y: " + str(y.shape))
print("length of y: " + str(len(y)))
print("value of y: " + str(y[0]))

In [ ]:
#split X and y into train and test
from sklearn.model_selection import train_test_split
X_train,X_test, y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=0) 

In [ ]:
print("shape of X_train: " + str(X_train.shape))
print("shape of y_train: " + str(y_train.shape))
print("shape of X_test: " + str(X_test.shape))
print("shape of y_test: " + str(y_test.shape))

In [ ]:
y_train

In [ ]:
print('Saving X_train')
np.save('X_train.npy', X_train)
print('Saving y_train')
np.save('y_train.npy', y_train)
print('Saving X_test')
np.save('X_test.npy', X_test)
print('Saving y_test')
np.save('y_test.npy', y_test)

In [ ]:
X_train = np.load('X_train.npy)
y_train = np.load('y_train.npy)
X_test = np.load('X_test.npy)
y_test = np.load('y_test.npy)

In [ ]:
from keras.utils.np_utils import to_categorical
y_train = to_categorical(y_train, 5)
y_test = to_categorical(y_test, 5)

In [ ]:
from keras.callbacks import ReduceLROnPlateau , ModelCheckpoint , LearningRateScheduler
lr_reduce = ReduceLROnPlateau(monitor='val_acc', factor=0.1, epsilon=0.0001, patience=1, verbose=1)

In [ ]:
filepath="tf_weightsv1-0.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')

In [ ]:
from keras.models import Sequential , Model
from keras.layers import Dense , Activation
from keras.layers import Dropout , GlobalAveragePooling2D
from keras.layers import Flatten
from keras.constraints import maxnorm
from keras.optimizers import SGD , RMSprop , Adadelta , Adam
from keras.layers import Conv2D , BatchNormalization
from keras.layers import MaxPooling2D
from keras.utils import np_utils
from keras import backend as K
K.set_image_dim_ordering('th')
from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasClassifier

In [ ]:
#use for channels last
#X_train=X_train.reshape(5216,250,250,3)
X_train=X_train.reshape(7898,250,250,3)
X_test=X_test.reshape(624,250,250,3)
K.set_image_data_format('channels_last')

In [ ]:
from keras.applications.inception_v3 import InceptionV3
# create the base pre-trained model
base_model = InceptionV3(weights=None, include_top=False , input_shape=(250, 250, 3))

In [ ]:
x = base_model.output
x = Dropout(0.5)(x)
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
#x = Dropout(0.5)(x)#new
#x = Dense(128, activation='relu')(x)#new
#x = Dropout(0.5)(x)#new
x = BatchNormalization()(x)
#x = BatchNormalization(axis=3)(x)#use for channels last
#predictions = Dense(2, activation='sigmoid')(x)
predictions = Dense(2, activation='softmax')(x)

In [ ]:
model = Model(inputs=base_model.input, outputs=predictions)

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])

In [ ]:
print(model.summary())

In [ ]:
batch_size = 64
epochs = 10

In [ ]:
history = model.fit(X_train, y_train, validation_data = (X_test , y_test),
                    callbacks=[lr_reduce,checkpoint], epochs=epochs)

In [ ]:
import pickle
with open('trainHistory1.pickle', 'wb') as file_history:
    pickle.dump(history.history, file_history)

In [ ]:
#from keras.models import load_model
#model = load_model('tf_weightsv1-0.hdf5')

In [ ]:
#model.load_weights("tf_weightsv1-0.hdf5")

In [ ]:
#with open(r"trainHistory.pickle", "rb") as input_file: history = Pickle.load(input_file)

In [ ]:
import matplotlib.pyplot as plt
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
from sklearn.metrics import confusion_matrix
pred = model.predict(X_test)
#pred = np.argmax(pred,axis = 1) 
#y_true = np.argmax(y_test,axis = 1)
y_true = y_test

In [ ]:
CM = confusion_matrix(y_true, pred)
from mlxtend.plotting import plot_confusion_matrix
fig, ax = plot_confusion_matrix(conf_mat=CM, figsize=(5, 5))
plt.show()

In [ ]:
precision = 390 / (390 + 137)
print(precision)

In [ ]:
recall = 390 / (390 + 0)
print(recall)

In [ ]:
f1 = (2*recall*precision) / (recall + precision)
print(f1)

In [4]:
x_validate = X_test[-5:]

NameError: name 'X_test' is not defined

In [ ]:
from keras.models import load_model
model2 = load_model('tf_weightsv1-0.hdf5')

In [ ]:
predictions = model2.predict(x_validate)

In [ ]:
predictions

In [ ]:
actual = y_test[-5:]
actual